<a href="https://colab.research.google.com/github/shown5/HandsOnLLM/blob/main/chap12_generative_model_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install -q accelerate==0.31.0 peft==0.11.1 bitsandbytes==0.43.1 transformers==4.41.2 trl==0.9.4 sentencepiece==0.2.0 triton==3.1.0 datasets==2.20.0

12.3 QLoRA による指示チューニング

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

# チャットテンプレートを使用するためのトークナイザーを読み込む
template_tokenizer = AutoTokenizer.from_pretrained(
    "TinyLlama/TinyLlma-1.1BChat-v1.0"
)

def format_prompt(example):
  """TinyLLmaが使用している<|user|>テンプレートを用いてプロンプトをフォーマットする"""

  # 回答をフォーマットする
  chat = example["messages"]
  prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)

  return {"text": prompt}

# データセットを読み込み、フォーマットする
dataset = (
    load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")
    .shuffle(seed=42)
    .select(range(3_000))
)
dataset = dataset.map(format_prompt)

In [ ]:
# フォーマットされたテンプレートの例
print(dataset["text"][2576])